# Assignment 3 on Natural Language Processing

## Date : 23rd October, 2021

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Prithwish Jana, Udit Desai

The central idea of this assignment is to experiment  Name Entity Recogniztion using pretrained language model BERT. Please split the NER data with train and validation, and evaluate the model on the validation.

Please submit with outputs. Submissions without predicted outputs will be penalized. **Please install necessary packages according to the task.**

In [47]:
import numpy as np 
import pandas as pd 


import os
#print(os.listdir("../input"))
!pip install pytorch-pretrained-bert
!pip install seqeval

# Any results you write to the current directory are saved as output.

Read the data

In [49]:
'''from google.colab import drive
drive.mount('/content/drive')
dataset = "/content/drive/MyDrive/Acads/A3_dataset.csv"
dframe = pd.read_csv(dataset, encoding = "ISO-8859-1", error_bad_lines=False)'''
dframe = pd.read_csv("A3_dataset.csv", encoding = "ISO-8859-1", error_bad_lines=False)

b'Skipping line 281837: expected 25 fields, saw 34\n'


In [50]:
dframe.head()

,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,pos,prev-iob,prev-lemma,prev-pos,prev-prev-iob,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,NNS,__START1__,__start1__,__START1__,__START2__,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,IN,O,thousand,NNS,__START1__,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,NNS,O,of,IN,O,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,VBP,O,demonstr,NNS,O,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,VBN,O,have,VBP,O,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [51]:
dataset=dframe.drop(['Unnamed: 0', 'lemma', 'next-lemma', 'next-next-lemma', 'next-next-pos',
       'next-next-shape', 'next-next-word', 'next-pos', 'next-shape',
       'next-word', 'prev-iob', 'prev-lemma', 'prev-pos',
       'prev-prev-iob', 'prev-prev-lemma', 'prev-prev-pos', 'prev-prev-shape',
       'prev-prev-word', 'prev-shape', 'prev-word','shape'],axis=1)

#dataset = dframe.dropdframe.drop([])

In [52]:
dataset.head()

,pos,sentence_idx,word,tag
0,NNS,1.0,Thousands,O
1,IN,1.0,of,O
2,NNS,1.0,demonstrators,O
3,VBP,1.0,have,O
4,VBN,1.0,marched,O


Preprocess the dataset

In [53]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w,p, t) for w,p, t in zip(s["word"].values.tolist(),
                                                       s['pos'].values.tolist(),
                                                        s["tag"].values.tolist())]
        self.grouped = self.dataset.groupby("sentence_idx").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [54]:
getter = SentenceGetter(dataset)

In [55]:
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country . Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [56]:
labels = [[s[2] for s in sent] for sent in getter.sentences]
print(labels[0])

['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


In [57]:
tags_vals = list(set(dataset["tag"].values))
tag2idx = {t: i for i, t in enumerate(tags_vals)}

Import model pacakages

In [58]:
import torch
import random
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from tqdm import tqdm, trange

Set up seed value

In [59]:
# setup random seed
seed =42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

In [60]:
MAX_LEN = 40
bs =64

In [61]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [62]:
torch.cuda.get_device_name(0) 

'Tesla K80'

Features generation  for pre-trained languge BERT

In [63]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print(tokenized_texts[0])

['thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'london', 'to', 'protest', 'the', 'war', 'in', 'iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'british', 'troops', 'from', 'that', 'country', '.', 'thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'london', 'to', 'protest', 'the', 'war', 'in', 'iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'british', 'troops', 'from', 'that', 'country', '.']


In [64]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [65]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

In [66]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [67]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [68]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [69]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [70]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))

In [71]:
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
      

In [72]:
param_optimizer = list(model.classifier.named_parameters()) 
optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [73]:
from seqeval.metrics import accuracy_score
import tensorflow as tf

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def model_validation_accuracy(model):
  model.eval()
  predictions,true_labels,accuracy_batch_wise= [], [] , []

  for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    #Using the above created flat accuracy function
    accuracy_batch_wise.append(flat_accuracy(logits,label_ids))
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.extend(label_ids)
  return predictions,true_labels,accuracy_score(predictions, true_labels)

In [74]:
epochs = 5
max_grad_norm = 1.0
validation_accuracy,epoch_total_loss,loss_per_epoch = [] , [], []

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    train_total_loss = 0
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        model.zero_grad()
        get_model = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        #get the loss
        loss = get_model
        # backward pass
        loss.backward()
        # track train loss
        train_total_loss += loss.item()
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        ## update parameters
        optimizer.step()
    __,___,validation_acc = model_validation_accuracy(model)
    # print train loss per epoch
    avg_train_loss = train_total_loss / len(train_dataloader)
    print("\nTotal Loss: {}".format(train_total_loss))
    print("Average train loss: {}".format(avg_train_loss))
    validation_accuracy.append(validation_acc)
    epoch_total_loss.append(train_total_loss)
    loss_per_epoch.append(avg_train_loss)
    print()
    #save your model after the final epoch
    #Saving after every epoch (incase of any issues)
    '''path = F"/content/NLP_Assignment3_Model_Epochs.pt" 
    torch.save(model.state_dict(), path)'''

Epoch:  20%|██        | 1/5 [07:42<30:51, 462.85s/it]


Total Loss: 892.4201981425285
Average train loss: 1.8028690871566233



Epoch:  40%|████      | 2/5 [15:26<23:09, 463.22s/it]


Total Loss: 419.5758949518204
Average train loss: 0.8476280706097381



Epoch:  60%|██████    | 3/5 [23:11<15:28, 464.30s/it]


Total Loss: 345.06359255313873
Average train loss: 0.6970981667740176



Epoch:  80%|████████  | 4/5 [31:03<07:47, 467.01s/it]


Total Loss: 320.8356241583824
Average train loss: 0.6481527760775402



Epoch: 100%|██████████| 5/5 [38:55<00:00, 467.05s/it]


Total Loss: 309.46551182866096
Average train loss: 0.6251828521791131



In [78]:
predictions, true_labels, final_validation_accuracy = model_validation_accuracy(model)
n_epoch = 5
print("- "*10,"Summary of the training epochs"," -"*10)
print()
for epoch in range(n_epoch):
  print("\033[4mFor Epoch",epoch,":\033[0m")
  print("Validation Accuracy:{}".format(validation_accuracy[epoch]))
  print("Total Loss: {}".format(epoch_total_loss[epoch]))
  print("Average train loss: {}".format(loss_per_epoch[epoch]))
  print()
print("\033[1mFinal Validation Accuracy of the Model: {}\033[0m".format(final_validation_accuracy))

- - - - - - - - - -  Summary of the training epochs  - - - - - - - - - -

For Epoch 0 :
Validation Accuracy:0.8946631608868676
Total Loss: 892.4201981425285
Average train loss: 1.8028690871566233

For Epoch 1 :
Validation Accuracy:0.8946702671972712
Total Loss: 419.5758949518204
Average train loss: 0.8476280706097381

For Epoch 2 :
Validation Accuracy:0.8945423536100057
Total Loss: 345.06359255313873
Average train loss: 0.6970981667740176

For Epoch 3 :
Validation Accuracy:0.8941586128482092
Total Loss: 320.8356241583824
Average train loss: 0.6481527760775402

For Epoch 4 :
Validation Accuracy:0.8938175099488346
Total Loss: 309.46551182866096
Average train loss: 0.6251828521791131

Final Validation Accuracy of the Model: 0.8938175099488346


In [80]:
tagged_values = list(set(dataset["tag"].values))
tagged_values.append("PAD")
tag_idx = {t: i for i, t in enumerate(tagged_values)}

predicted_tags = [tagged_values[p_i] for p, l in zip(predictions, true_labels)
                              for p_i, l_i in zip(p, l) if tagged_values[l_i] != "PAD"]
valid_tags = [tagged_values[l_i] for l in true_labels
                              for l_i in l if tagged_values[l_i] != "PAD"]
df_actual_pred=pd.DataFrame({"Predicted Tags":predicted_tags,"Actual Tags":valid_tags})
print(df_actual_pred)

       Predicted Tags Actual Tags
0                   O           O
1                   O           O
2                   O           O
3                   O           O
4                   O           O
...               ...         ...
140715              O           O
140716              O           O
140717              O           O
140718              O           O
140719              O           O

[140720 rows x 2 columns]


In [81]:
#Save the  model and result
df_actual_pred.to_csv('/content/19CH30018_NLP_Assignment3_Output.csv',index=False)
path = F"/content/19CH30018_NLP_Assignment3_Final_Model_Epochs.pt" 
torch.save(model.state_dict(), path)